In [ ]:
# converting model into tflite, we will use tflite model to deploy in edge device

In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import os

In [2]:
path_of_saved_model = Path("./trained_model.h5")
model = tf.keras.models.load_model(path_of_saved_model)

In [3]:

path_of_saved_model

WindowsPath('trained_model.h5')

In [4]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\jagan\AppData\Local\Temp\tmpzaasae10\assets


In [6]:
tflite_model_dir = Path("./tflite_model_dir")
tflite_model_dir.mkdir(exist_ok=True, parents=True)

In [7]:
tflite_model_dir/"vgg16_model.tflite"


WindowsPath('tflite_model_dir/vgg16_model.tflite')

In [8]:
tflite_model_file = tflite_model_dir/"vgg16_model.tflite"
tflite_model_file.write_bytes(tflite_model)  # save the model into bytes format

59067384

In [ ]:
# even after converting into the tflite format, it is still 57 MB size.
# we need to make it lite weight, so we will use optimizations 

In [9]:

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

tflite_model_quant_file = tflite_model_dir/"vgg16_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\jagan\AppData\Local\Temp\tmpys4dvg8k\assets


INFO:tensorflow:Assets written to: C:\Users\jagan\AppData\Local\Temp\tmpys4dvg8k\assets


14870960

In [ ]:
# now size of the model reduced to 15 MB

In [10]:
14870960/59067384  # model size is reduced by 25%


0.25176263096398516

In [ ]:
# lets verify model is working fine or not.. try with prediction
# refer
# https://www.tensorflow.org/lite/guide/inference

#Running a model
# Running a TensorFlow Lite model involves a few simple steps:
# Load the model into memory.
# Build an Interpreter based on an existing model.
# Set input tensor values. (Optionally resize input tensors if the predefined sizes are not desired.)
# Invoke inference.
# Read output tensor values.

In [12]:
#loading tflite full model
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()  # allocating tensors into the memory

In [13]:
#loading tflite quant model
interpreter_quant = tf.lite.Interpreter(
    model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [11]:
import matplotlib.pyplot as plt
test_img = plt.imread(
    "./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")


In [14]:
test_img.shape


(534, 800, 3)

In [16]:
interpreter.get_input_details()


[{'name': 'serving_default_input_2:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [15]:
interpreter.get_input_details()[0]["shape"]

array([  1, 224, 224,   3])

In [18]:

resized_img = tf.image.resize(test_img, (224, 224))
resized_img.shape

TensorShape([224, 224, 3])

In [20]:

input_data = tf.expand_dims(resized_img, axis=0)
input_data.shape  # [batch_size, width, height, channel]

TensorShape([1, 224, 224, 3])

In [22]:
interpreter.get_input_details()


[{'name': 'serving_default_input_2:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [26]:
input_index = interpreter.get_input_details()[0]["index"]

In [23]:
interpreter.get_output_details()


[{'name': 'StatefulPartitionedCall:0',
  'index': 50,
  'shape': array([1, 2]),
  'shape_signature': array([-1,  2]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [27]:
output_index = interpreter.get_output_details()[0]["index"]


In [28]:

# invoking and setting the tensors
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)

In [29]:
pred

array([[1.545303e-20, 1.000000e+00]], dtype=float32)

In [30]:
label_map = {0: 'ants', 1: 'bees'}

In [31]:
argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'bees'

In [32]:
# similarly to the above steps, we will do for 'interpreter_quant'
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)

argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]


'bees'